In [1]:
# Python Dependencies
import pandas as pd
import numpy as np
from pathlib import Path


In [2]:
# Import findspark and initialize. 
import findspark
findspark.init()

In [3]:
# Import packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date,when
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# Store filepath in a variable
constituents_path = Path("./Resources/constituents.csv")

In [5]:
# 1. Read CSV file 
constituents_df = spark.read.csv(str(constituents_path), sep=",", header=True)
# Show the dataframe
constituents_df.show()

+-------------+-----------+-----+----------+-------------+----------------+--------------------+---------------+-----------------+---------------+---------------+--------------------+------------------+----------------+----------------+--------------------+--------------------+------------------+------------------+--------------------+----------+----+-----------+--------------------+--------------------+------+--------------+--------------------+---------------------+----+-----------------------+--------------------+---------------------+-----------------+
|ConstituentID|       City|State|  Postcode|      Country|Constituent type|   Constituent codes|Lifetime giving|First gift amount|First gift type|First gift date|     First gift fund|Latest gift amount|Latest gift type|Latest gift date|    Latest gift fund|Greatest gift amount|Greatest gift type|Greatest gift date|  Greatest gift fund| Birthdate| Age|AgeInMonths|   PrimaryEmployment|           Education|Gender|Marital status|       S

In [6]:
constituents_df.columns

['ConstituentID',
 'City',
 'State',
 'Postcode',
 'Country',
 'Constituent type',
 'Constituent codes',
 'Lifetime giving',
 'First gift amount',
 'First gift type',
 'First gift date',
 'First gift fund',
 'Latest gift amount',
 'Latest gift type',
 'Latest gift date',
 'Latest gift fund',
 'Greatest gift amount',
 'Greatest gift type',
 'Greatest gift date',
 'Greatest gift fund',
 'Birthdate',
 'Age',
 'AgeInMonths',
 'PrimaryEmployment',
 'Education',
 'Gender',
 'Marital status',
 'Solicit codes',
 'Student Organizations',
 'Arts',
 'Fraternal Organizations',
 "Men's Athletics",
 'Scholarship Recipient',
 "Women's Athletics"]

In [7]:
constituents_df.printSchema()

root
 |-- ConstituentID: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Constituent type: string (nullable = true)
 |-- Constituent codes: string (nullable = true)
 |-- Lifetime giving: string (nullable = true)
 |-- First gift amount: string (nullable = true)
 |-- First gift type: string (nullable = true)
 |-- First gift date: string (nullable = true)
 |-- First gift fund: string (nullable = true)
 |-- Latest gift amount: string (nullable = true)
 |-- Latest gift type: string (nullable = true)
 |-- Latest gift date: string (nullable = true)
 |-- Latest gift fund: string (nullable = true)
 |-- Greatest gift amount: string (nullable = true)
 |-- Greatest gift type: string (nullable = true)
 |-- Greatest gift date: string (nullable = true)
 |-- Greatest gift fund: string (nullable = true)
 |-- Birthdate: string (nullable = true)
 |-- Age: string (nullable 

In [8]:
#Change amounts coulumns to decimals 
constituents_df = constituents_df.withColumn("Lifetime giving", col("Lifetime giving").cast("decimal"))
constituents_df = constituents_df.withColumn("First gift amount", col("First gift amount").cast("decimal"))
constituents_df = constituents_df.withColumn("Latest gift amount", col("Latest gift amount").cast("decimal"))
constituents_df = constituents_df.withColumn("Greatest gift amount", col("Greatest gift amount").cast("decimal"))


In [13]:
# Change to date format
constituents_df = constituents_df.withColumn("First gift date", to_date(col("First gift date"), "MM/dd/yyyy"))
#constituents_df = constituents_df.withColumn("Birthdate", to_date(col("Birthdate"), "MM-dd-yyyy"))
#constituents_df = constituents_df.withColumn("Latest gift date", to_date(col("Latest gift date"), "MM-dd-yyyy"))
#constituents_df = constituents_df.withColumn("Greatest gift date", to_date(col("Greatest gift date"), "MM-dd-yyyy"))

In [17]:
#Add new column for repeated 
constituents_df = constituents_df.withColumn('repeatedConstituents', when(col('Lifetime giving') > col('First gift amount'), 'Yes').otherwise('No'))

In [18]:
#Create a temporary view of the DataFrame.
constituents_df.createOrReplaceTempView('constituents')

In [19]:
# Check the view data
spark.sql("""SELECT 
            `ConstituentID`,`Lifetime giving`,
             `First gift amount`,`First gift date`,
             `Latest gift amount`,`Latest gift date`,
             `Greatest gift amount`,`Greatest gift date`,
             repeatedConstituents
             
   FROM 
       constituents 
   WHERE 
   repeatedConstituents='Yes'
    
   """).show()

+-------------+---------------+-----------------+---------------+------------------+----------------+--------------------+------------------+--------------------+
|ConstituentID|Lifetime giving|First gift amount|First gift date|Latest gift amount|Latest gift date|Greatest gift amount|Greatest gift date|repeatedConstituents|
+-------------+---------------+-----------------+---------------+------------------+----------------+--------------------+------------------+--------------------+
|        36583|            450|               50|           NULL|               200|      12/29/2023|                 200|        12/29/2023|                 Yes|
|        21368|            600|              100|           NULL|               500|      12/19/1985|                 500|        12/19/1985|                 Yes|
|        44089|            350|              300|           NULL|               250|       4/10/2024|                 300|        10/25/2022|                 Yes|
|        39798|       

In [13]:
#Count distinct donors 
spark.sql("""SELECT 
            count(distinct(`ConstituentID`))
   FROM 
       constituents 
    
   """).show()

+-----------------------------+
|count(DISTINCT ConstituentID)|
+-----------------------------+
|                        50913|
+-----------------------------+



In [14]:
#count the number of donors whos has not contribute yet 
spark.sql("""SELECT 
            count(distinct(`ConstituentID`))
   FROM 
       constituents 
   WHERE 
    `First gift amount`=0
    
   """).show()

+-----------------------------+
|count(DISTINCT ConstituentID)|
+-----------------------------+
|                            0|
+-----------------------------+



In [15]:
#Repeated donors
spark.sql("""SELECT 
        count(*)
   FROM 
       constituents 
   WHERE 
   `repeatedConstituents`='Yes'
   """).show()


+--------+
|count(1)|
+--------+
|    4358|
+--------+



In [16]:
#Null information among the active donors
spark.sql("""
   SELECT 
      COUNT(`ConstituentID`)
   FROM 
       constituents 
   WHERE 
      `First gift amount` > 0 AND City IS NULL

""").show()

+--------------------+
|count(ConstituentID)|
+--------------------+
|                 150|
+--------------------+



In [17]:
#Group records as per the funds 
spark.sql("""
   SELECT 
      `First gift fund`,count(`ConstituentID`) as totConstituent
   FROM 
       constituents 
   WHERE 
      `First gift amount` > 0 
  GROUP BY  `First gift fund`

""").show()

+--------------------+--------------+
|     First gift fund|totConstituent|
+--------------------+--------------+
|Women's Basketbal...|           310|
|Business and Econ...|            33|
|Multicultural Cen...|            10|
|Mary Jo Gangnon M...|             9|
|General Charles C...|             8|
|0-Francis C DeVin...|             2|
|Davis, John Bloom...|            46|
|Robert Banks Memo...|            11|
|     Mock Trial Fund|            37|
|Superior Voices S...|             5|
| Women's Tennis Fund|            64|
|Everson, Dr. and ...|             6|
|Boswell, Melinda,...|             1|
|0-Center Court Cl...|            14|
|Distance Learning...|             9|
| Women's Hockey Fund|           144|
|Grad Psych Develo...|             1|
|SCCU Stadium Dan ...|            13|
|Brown, Maurice, S...|             4|
|Art Student Leagu...|             2|
+--------------------+--------------+
only showing top 20 rows



In [18]:
#Convert to Pandas dataframe
constituents_pd_df = constituents_df.toPandas()
constituents_pd_df.head()

,ConstituentID,City,State,Postcode,Country,Constituent type,Constituent codes,Lifetime giving,First gift amount,First gift type,...,Gender,Marital status,Solicit codes,Student Organizations,Arts,Fraternal Organizations,Men's Athletics,Scholarship Recipient,Women's Athletics,repeatedConstituents
0,60472,Superior,WI,54880-2556,United States,Organization,Business (No start date - No end date),0,None,None,...,None,None,None,None,None,None,None,None,None,No
1,43735,Hermantown,MN,55811-1755,United States,Organization,Other Organizations (8/29/2012 - No end date),50,50,One-time gift,...,None,None,None,None,None,None,None,None,None,No
2,60145,Esko,MN,55733-9645,United States,Organization,Other Organizations (No start date - No end date),50,50,One-time gift,...,None,None,None,None,None,None,None,None,None,No
3,21332,None,None,None,United States,Organization,Business (No start date - No end date),25,25,One-time gift,...,None,None,None,None,None,None,None,None,None,No
4,50410,Superior,WI,54880-1504,United States,Organization,Business (No start date - No end date),0,None,None,...,None,None,None,None,None,None,None,None,None,No


In [19]:
# Convert the age of donors & first donation date to date format
# Calculate the ages at time of first donation 
